This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
# Lib imports
import os
import tensorflow as tf
# Use the ImageDataGenerator from the tensorflow runtime to avoid static import resolution issues
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
# Access keras submodules via the tf object to avoid static import resolution issues
layers = tf.keras.layers
models = tf.keras.models
import numpy as np

In [2]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "./train_catdog"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "./test_catdog"    # e.g. './muffin-vs-chihuahua/test'

In [3]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 10002 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Apply L2 regularization and stronger dropout (SpatialDropout2D for conv blocks)
l2_reg = tf.keras.regularizers.l2(1e-4)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2_reg, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.SpatialDropout2D(0.2),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2, 2),
    layers.SpatialDropout2D(0.2),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2, 2),
    layers.SpatialDropout2D(0.2),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=l2_reg),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

c:\Users\ledes\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

print(train_generator.class_indices)

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 126s 399ms/step - accuracy: 0.5158 - loss: 0.7201 - val_accuracy: 0.5648 - val_loss: 0.7061
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 126s 399ms/step - accuracy: 0.5158 - loss: 0.7201 - val_accuracy: 0.5648 - val_loss: 0.7061
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 53s 168ms/step - accuracy: 0.5309 - loss: 0.7024 - val_accuracy: 0.5832 - val_loss: 0.6742
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 53s 168ms/step - accuracy: 0.5309 - loss: 0.7024 - val_accuracy: 0.5832 - val_loss: 0.6742
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 49s 157ms/step - accuracy: 0.5841 - loss: 0.6852 - val_accuracy: 0.6416 - val_loss: 0.6737
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 49s 157ms/step - accuracy: 0.5841 - loss: 0.6852 - val_accuracy: 0.6416 - val_loss: 0.6737
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 52s 165ms/step - accuracy: 0.6201 - loss: 0.6664 - val_accuracy: 0.6780 - val_loss: 0.6184
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 52s 165ms/step - accuracy: 0.6201 - loss:

In [8]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 261ms/step - accuracy: 0.7145 - loss: 0.5648
Test Accuracy: 0.7145000100135803
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 261ms/step - accuracy: 0.7145 - loss: 0.5648
Test Accuracy: 0.7145000100135803


In [9]:
# SAVE THE MODEL
# model.save('muffin_vs_chihuahua_cnn.h5')
model.save('exercise_6_catdog_ledesma.h5')

In [10]:
# SIMPLE INFERENCE SCRIPT
# Use the tensorflow runtime object 'tf' already imported in the notebook
# to access the preprocessing utilities to avoid static import resolution issues.
image = tf.keras.preprocessing.image

def predict_image(img_path, model_path='exercise_6_catdog_ledesma.h5'):
    model = tf.keras.models.load_model(model_path, compile=False)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    
    # Updated for cat vs dog classification
    label = "Dog" if pred >= 0.5 else "Cat"
    print(f"Prediction: {label} (confidence: {pred:.2f})")
    
    # Old muffin vs chihuahua version:
    # label = "Chihuahua" if pred >= 0.5 else "Muffin"
    # print(f"Prediction: {label} (confidence: {pred:.2f})")

In [12]:
# Example usage:
predict_image("./predict/predict_cat.jpg")
predict_image("./predict/predict_dog.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Prediction: Dog (confidence: 0.58)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Prediction: Dog (confidence: 0.82)
Prediction: Dog (confidence: 0.58)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Prediction: Dog (confidence: 0.82)
